In [17]:
!python3 -V

Python 3.11.0rc1


In [18]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [19]:
#Question 1 : Download data and calculate column number.
#download data 
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
#df = read_dataframe('yellow_tripdata_2023-01.parquet')
total_rows=len(df)

In [20]:
#Calculate column number
print(len(df.columns))
df.head()

19


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [21]:
#Question2: Computing Duration
#create column duration
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
round(df['duration'].std(),2)


42.59

In [22]:
#Question 3: removing outliers outside 1 to 60 minutes.

df = df[(df.duration >= 1) & (df.duration <= 60)]
new_rows=len(df)
print(total_rows)
print(new_rows)

percent_rem_out = (total_rows - new_rows) / total_rows * 100 
print(100-percent_rem_out)

3066766
3009173
98.1220282212598


In [23]:
#Question 4: One hot decoding
categorical = ['PULocationID', 'DOLocationID']
#numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape[1]

515

In [24]:
#Question 5: Training the model with linear regression

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

/home/nevin/GitHub/mlops-zoomcamp/.venv/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.649262223700304

In [25]:
#function to automate reading data and calculate the interval in minutes. 
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
        
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    
    return df

In [26]:
# downloading train and validation data
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')
len(df_train), len(df_val)

(3009173, 2855951)

In [27]:
df_train[categorical].dtypes

PULocationID    int64
DOLocationID    int64
dtype: object

In [28]:
#Question 6: Calculate feature matrix with 2 features
categorical = ['PULocationID', 'DOLocationID']

dv = DictVectorizer()
#turn train data it to dictionaries
df_train[categorical] = df_train[categorical].astype(str)
train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

#turn validation into dictionaries 
df_val[categorical] = df_val[categorical].astype(str)
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)


target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

#fit linear regression model to train our data
lr = LinearRegression()
lr.fit(X_train, y_train)

#our target is predict duration values
y_pred = lr.predict(X_val)

#check predicted target values with validation data 
mean_squared_error(y_val, y_pred, squared=False)

/home/nevin/GitHub/mlops-zoomcamp/.venv/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.811813608870425